# MDL DB roll decay content
What is the data in the roll decay database and roll damping database?

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)
import numpy as np
import os
import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['figure.figsize'] = 15, 5

import data
import copy
from mdldb.mdl_db import MDLDataBase
from mdldb.tables import Base, Model, LoadingCondition, Run, RolldecayQuadraticB

from rolldecayestimators.direct_estimator import DirectEstimator
from rolldecayestimators.direct_linear_estimator import DirectLinearEstimator
from rolldecayestimators.norwegian_estimator import NorwegianEstimator
from rolldecayestimators.transformers import CutTransformer, LowpassFilterDerivatorTransformer, ScaleFactorTransformer, OffsetTransformer
#from rolldecay.equations_lambdify import calculate_acceleration, calculate_velocity
from sklearn.pipeline import Pipeline

import signal_lab
from sqlalchemy.inspection import inspect
import seaborn as sns
import docs

In [ ]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///' + data.mdl_db_path)
db = MDLDataBase(engine=engine)

In [ ]:
roll_decay_tests = db.session.query(Run).filter(
                                                (Run.test_type=='roll decay'))

In [ ]:
len(roll_decay_tests.all())

In [ ]:
ships = db.session.query(Ship).filter((Run.test_type=='roll decay'))

In [ ]:
len(ships.all())

In [ ]:
df_ships = pd.DataFrame()

for ship in ships:
    instance = inspect(ship)
    
    s = pd.Series()
                
    for key,item in instance.attrs.items():
        if not isinstance(item.value, list):
            if pd.notnull(item.value):
                s[key]=item.value
    
    df_ships = df_ships.append(s, ignore_index=True)

In [ ]:
mask = df_ships[['lpp','beam','BKL','BKB']].notnull().all(axis=1)
df_ships = df_ships.loc[mask].copy()

In [ ]:
df_ships.hist(column = ['lpp','beam','BKL','BKB'],bins=20)

In [ ]:
len(df_ships)